# **Pandas_Functions**

In [ ]:
import sys
sys.path.append("..")
import suntzu # noqa
df = suntzu.Loading.read_file("Titanic.csv")

In [ ]:
suntzu.PreProcessing.get_dims(df)

In [ ]:
def set_full_view(max_cols="yes", max_rows="yes", max_width="yes", max_colwidht="yes"):
   from pandas import set_option
   if max_cols == "yes":
      set_option('display.max_rows', None)
   elif max_rows == "yes":
      set_option('display.max_columns', None)
   elif max_width == "yes":
      set_option('display.width', None)
   elif max_colwidht == "yes":
      set_option('display.max_colwidth', None)

File Variables

In [3]:
def print_file_variables(self):
    variables = list(self.variables.keys())
    print(variables)

In [ ]:
def get_file_variables(self):
    variables = list(self.variables.keys())
    return variables

In [ ]:
import pandas as pd #noqa
import polars as pl #noqa
import xarray as xr #noqa
def check_dataframe_type(self):
    if isinstance(self, pd.DataFrame):
        print("This is a Pandas DataFrame.")
    elif isinstance(self, pl.DataFrame):
        print("This is a Polars DataFrame.")
    elif isinstance(self, xr.Dataset):
        print("This is a xarray Dataset")
    else:
        print("Unknown DataFrame/Dataset type.")

# Metadata

## netCDF

### Get

In [34]:
def get_netCDF_metadata(self, variables=None, output_format="separate"):
    """
    Get metadata for variables.

    Args:
        variables (list): List of variable names. If None, metadata for all variables will be retrieved.
        output_format (str): Output format for variable metadata. Valid options are 'separate' and 'together'.
    Returns:
        str: The metadata for the variables.
    """
    def get_variable_metadata(var_name, variables, separate=True):
        """
        Get metadata for a variable.

        Args:
            var_name (str): Name of the variable.
            var (Variable): Variable object.
            separate (bool): Whether to return the metadata separately or together.

        Returns:
            str: The metadata for the variable.
        """
        metadata = f"Variable: {var_name}\n"
        if not variables.attrs:
            metadata += "   No metadata was found for this variable.\n"
            if variables.values is not None:
                metadata += f"   Values: {variables.values}\n"
        else:
            metadata += f"   Values: {variables.values}\n"
            metadata += f"   Attributes: {variables.attrs}\n"
        return metadata if separate else metadata.strip()

    if variables is None:
        if not self.coords:
            variables = []
    else:
        if not variables:
            metadata = ""
            for coord_name, coord_var in self.coords.items():
                try:
                    metadata += get_variable_metadata(coord_name, coord_var, separate=(output_format=="separate")) + "\n"
                except KeyError:
                    metadata += f"Variable {coord_name} not found.\n"
            return metadata
        else:
            metadata = ""
            for var_name in variables:
                try:
                    coord_var = self.coords[var_name]
                    metadata += get_variable_metadata(var_name, coord_var, separate=(output_format=="separate")) + "\n"
                except KeyError:
                    metadata += f"Variable {var_name} not found.\n"
            return metadata

In [ ]:
def get_global_metadata(self):
    """
    Returns a string representation of the global metadata attributes.
    If no global attributes are found, returns "No Global Attributes were found."
    """
    global_attrs = self.attrs
    if global_attrs is None:
        return "No Global Attributes were found."
    else:
        metadata_str = ""
        for attr_name, attr_value in global_attrs.items():
            metadata_str += f"{attr_name}: {attr_value}\n"
        return metadata_str

### Print


In [63]:
def print_netCDF_metadata(self, variables=None, output_format="separate"):
    """
    Get metadata for variables.

    Args:
        variables (list): List of variable names. If None, metadata for all variables will be retrieved.
        output_format (str): Output format for printing variable metadata. Valid options are 'separate' and 'together'.
    Returns:
        None
    """
    def print_variable_metadata(var_name, var, separate=True):
        """
        Print metadata for a variable.

        Args:
            var_name (str): Name of the variable.
            var (Variable): Variable object.
            separate (bool): Whether to print the metadata separately or together.

        Returns:
            None
        """
        print("Variable:", var_name)
        if not var.attrs:
            print("   No attributes were found for this variable.")
            if var.values is not None:
                print("   Values: ", var.values)
        else:
            print(" Attributes:")
            for key, value in var.attrs.items():
                print(f"   {key}: {value}")
            print(" Values: ", var.values)
        print() if separate else print("   ", var)

    if variables is None:
        variables = list(self.coords.keys())
    for var_name in variables:
        try:
            coord_var = self.coords[var_name]
            print_variable_metadata(var_name, coord_var, separate=(output_format=="separate"))
        except KeyError:
            print(f"Variable {var_name} not found.")

In [46]:
def print_global_metadata(self):
    # Get the global attributes of the dataset
    global_attrs = self.attrs
    # Check if the global attributes are present
    if global_attrs is None:
        print("No Global Attributes were found.")
    else:
        # If there are, it will loop over each global attribute and print its name and value
        for attr_name, attr_value in global_attrs.items():
            title_attr_name = attr_name.title() # Improve readability
            print(title_attr_name, ":", attr_value)

### Write

In [12]:
def input_netCDF_metadata(self, variables=None, attributes=None, new_file="no", filename="new_file.nc"):
    # Define default attributes if not provided
    default_attributes = [
        "Units", "Long_Name", "Standard_Name/Short_Name", 
        "Valid_Min", "Valid_Max", "Missing_Value", 
        "Fill_Value", "Scale_Factor", "Add_Offset", 
        "Coordinates", "Axis", "Description"
    ]
    if attributes is None:
        attributes = default_attributes

    if variables is None:
        variables = list(self.coords.keys())

    for coord_name in variables:
        try:
            for attribute in attributes:
                self[coord_name].attrs[attribute] = input(f"{coord_name}: {attribute} - Enter value: ")
        except KeyError:
            print(f"Variable {coord_name} not found.")
    try:
        if new_file.lower() == "yes":
            self.to_netcdf(filename)
    except ValueError:
        print("")

In [57]:
input_netCDF_metadata(nc_dataset, ["lat", "lon"], ["Units", "Metadados"], "yes", filename="wdwad.nc")

In [64]:
teste = xr.open_dataset("wdwad.nc")
print_netCDF_metadata(teste)

Variable: lat
 Attributes:
   Units: a
   Metadados: a
 Values:  [-7.52       -7.51916667 -7.51833333 ... -5.56833333 -5.5675
 -5.56666667]

Variable: lon
 Attributes:
   Units: a
   Metadados: a
 Values:  [-79.49583333 -79.495      -79.49416667 ... -76.94666667 -76.94583333
 -76.945     ]

Variable: pair
   No attributes were found for this variable.
   Values:  2021-11-16 2021-11-28

Variable: ref
   No attributes were found for this variable.
   Values:  2021-11-16

Variable: rep
   No attributes were found for this variable.
   Values:  2021-11-28



In [3]:
print_netCDF_metadata(nc_dataset)

Variable: lat
   No metadata was found for this variable.
   Values:  [-7.52       -7.51916667 -7.51833333 ... -5.56833333 -5.5675
 -5.56666667]

Variable: lon
   No metadata was found for this variable.
   Values:  [-79.49583333 -79.495      -79.49416667 ... -76.94666667 -76.94583333
 -76.945     ]

Variable: pair
   No metadata was found for this variable.
   Values:  2021-11-16 2021-11-28

Variable: ref
   No metadata was found for this variable.
   Values:  2021-11-16

Variable: rep
   No metadata was found for this variable.
   Values:  2021-11-28



## Parquet

In [5]:
import pyarrow.parquet as pq
df = pq.read_table("updated_Titanic.parquet")

# Testing

In [2]:
import xarray as xr #noqa
nc_dataset = xr.open_dataset("example.nc")
#print_file_variables(nc_dataset)
#get_variables_metadata(nc_dataset)

In [7]:
nc_dataset.head()

<xarray.Dataset>
Dimensions:  (lat: 5, lon: 5)
Coordinates:
  * lat      (lat) float64 -7.52 -7.519 -7.518 -7.518 -7.517
  * lon      (lon) float64 -79.5 -79.49 -79.49 -79.49 -79.49
    pair     object ...
    ref      object ...
    rep      object ...
Data variables:
    los      (lat, lon) float32 ...

In [12]:
print_netCDF_metadata(nc_dataset)

Variable: lat
   No metadata was found for this variable.

Variable: lon
   No metadata was found for this variable.

Variable: pair
   No metadata was found for this variable.

Variable: ref
   No metadata was found for this variable.

Variable: rep
   No metadata was found for this variable.



In [47]:
print_global_metadata(nc_dataset)

In [ ]:
def modify_col_case(self, option=""):
    if option == "title":                             
        self = self.rename(columns=lambda x: x[0].title() + x[1:]).head()
    elif option =="upper":
        self = self.rename(columns=lambda x: x[0].upper() + x[1:]).head()
    else:
        self = self.rename(columns=lambda x: x[0].lower() + x[1:]).head()

### **Getting The Dimensions**

### Drop Columns